In [76]:
import fiona
from shapely.geometry import box
import folium
from shapely.geometry import Point
import os
import time
import pyogrio
import geopandas as gpd
import geopandas as gpd
import pyogrio
from shapely.geometry import Point
import folium
import pyogrio
import geopandas as gpd
import folium
from shapely.geometry import Point
from IPython.display import display

## Antall rader

In [16]:


fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output.fgb"
gdf = gpd.read_file(fgb_file)

print(gdf.geom_type.value_counts())  # Teller Polygon og MultiPolygon
print(len(gdf))  # Sjekker total antall rader


MultiPolygon    176214
Name: count, dtype: int64
176214


## Konvertering til FGB (GML og GDB)

In [80]:

def convert_ar50_to_fgb(input_file, output_file):
    """
    Konverterer AR50-data fra GML eller GDB til FlatGeobuf (FGB),
    og filtrerer kun ut polygon-geometrier.
    """
    file_ext = os.path.splitext(input_file)[1].lower()

    # Bruk riktig lag for hver filtype
    if file_ext == ".gml":
        layer_name = "ArealressursFlate"
        gdf = gpd.read_file(input_file, layer=layer_name)

    elif file_ext == ".gdb":
        layer_name = "ar50"
        if layer_name not in fiona.listlayers(input_file):
            raise ValueError(f"Laget '{layer_name}' finnes ikke i GDB-filen.")
        gdf = gpd.read_file(input_file, layer=layer_name)

    else:
        raise ValueError(f"Ukjent eller ikke støttet filtype: {file_ext}")

    # Filtrer kun polygoner
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]

    # Lagre som FlatGeobuf
    gdf.to_file(output_file, driver="FlatGeobuf")
    print(f"Lagret '{layer_name}' som {output_file}")

    return gdf

# Eksempelbruk
input_gml = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/42_25832_ar50_gml.gml"
input_gdb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/42_25832_ar50_gdb.gdb"

# Konverter GML til FGB
output_fgb_gml = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gml.fgb"
convert_ar50_to_fgb(input_gml, output_fgb_gml)

# Konverter GDB til FGB
output_fgb_gdb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gdb.fgb"
convert_ar50_to_fgb(input_gdb, output_fgb_gdb)


Lagret 'ArealressursFlate' som /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gml.fgb
Lagret 'ar50' som /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gdb.fgb


,rid,kilde,artype,arskogbon,artreslag,arjordbr,arveget,arkartstd,komid,fylkeid,geometry
0,1,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,52,AR50,4206,42,"MULTIPOLYGON (((357097.324 6444797.96, 357057...."
1,2,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,20,98,98,24,98,AR50,4206,42,"MULTIPOLYGON (((357232.182 6444717.404, 357231..."
2,3,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,55,AR50,4206,42,"MULTIPOLYGON (((357578.261 6444906.04, 357585...."
3,4,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,30,13,31,98,98,AR50,4206,42,"MULTIPOLYGON (((357998.132 6441329.147, 358005..."
4,5,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,60,11,39,98,98,AR50,4206,42,"MULTIPOLYGON (((358617.14 6442580.943, 358631...."
...,...,...,...,...,...,...,...,...,...,...,...
176209,176210,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,30,11,31,98,98,AR50,4212,42,"MULTIPOLYGON (((485458.32 6520535.79, 485484.5..."
176210,176211,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,60,11,39,98,98,AR50,4212,42,"MULTIPOLYGON (((483990.429 6519664.481, 483991..."
176211,176212,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,60,12,31,98,98,AR50,4212,42,"MULTIPOLYGON (((486986.352 6519317.402, 486986..."
176212,176213,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,30,11,99,98,98,AR50,4212,42,"MULTIPOLYGON (((485607.834 6519514.217, 485623..."


## Leser filen

In [83]:

# Sti til din FlatGeobuf (.fgb) fil
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"  # Endre til din filsti

# Les inn FGB-filen
gdf = gpd.read_file(fgb_file)

# Vis de 10 første radene som en tabell
gdf.head(10)


,kilde,artype,arskogbon,artreslag,arjordbr,arveget,arkartstd,geometry
0,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,99,98,98,98,98,AR50,"MULTIPOLYGON (((499330.268 6451352.944, 499395..."
1,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,99,98,98,98,98,AR50,"MULTIPOLYGON (((463337.546 6419930.693, 463427..."
2,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,99,98,98,98,98,AR50,"MULTIPOLYGON (((455377.855 6416267.632, 455468..."
3,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,82,98,98,98,98,AR50,"MULTIPOLYGON (((459362.812 6418093.267, 459272..."
4,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,99,AR50,"MULTIPOLYGON (((455999.361 6442165.662, 456015..."
5,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,51,AR50,"MULTIPOLYGON (((455969.733 6442883.25, 455964...."
6,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,52,AR50,"MULTIPOLYGON (((455842.562 6442434.745, 455822..."
7,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,54,AR50,"MULTIPOLYGON (((455599.141 6442677.93, 455612...."
8,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,51,AR50,"MULTIPOLYGON (((455429.774 6442400.392, 455427..."
9,AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...,50,98,39,98,52,AR50,"MULTIPOLYGON (((454851.4 6442824.725, 454871.8..."


## Gjør filen mindre

In [85]:


def reduce_ar50_file(input_fgb, output_fgb, columns_to_keep):
    """
    Leser en FGB-fil, fjerner uønskede kolonner, begrenser datasettet til et visst antall rader
    og lagrer det til en ny FGB-fil.

    Args:
        input_fgb (str): Sti til input FGB-fil.
        output_fgb (str): Sti til output FGB-fil.
        columns_to_keep (list): Liste med kolonner som skal beholdes.
        max_rows (int): Maks antall rader i det nye datasettet (default: 50).
    """
    # Les inn FlatGeobuf-filen
    gdf = gpd.read_file(input_fgb)

    # Behold kun de ønskede kolonnene
    gdf = gdf[columns_to_keep]

    # Lagre det nye datasettet
    gdf.to_file(output_fgb, driver="FlatGeobuf")

    print(f"Fil lagret som: {output_fgb}")
    print(f"Kolonner beholdt: {list(gdf.columns)}")
    print(f"Antall rader: {len(gdf)}")

# Eksempel på bruk
input_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gdb.fgb"
output_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
columns_to_keep = ['kilde', 'artype', 'arskogbon', 'artreslag', 'arjordbr', 'arveget', 'arkartstd', 'geometry']


reduce_ar50_file(input_fgb, output_fgb, columns_to_keep)


Fil lagret som: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb
Kolonner beholdt: ['kilde', 'artype', 'arskogbon', 'artreslag', 'arjordbr', 'arveget', 'arkartstd', 'geometry']
Antall rader: 176214


## Viser kolonnene i filen

In [86]:

input_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"

gdf = gpd.read_file(input_fgb)
print("Tilgjengelige kolonner:", gdf.columns.tolist())


Tilgjengelige kolonner: ['kilde', 'artype', 'arskogbon', 'artreslag', 'arjordbr', 'arveget', 'arkartstd', 'geometry']


## Simulert strømming med "sleep"

In [93]:

# Filbane til AR50 FlatGeobuf (FGB)
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"

# Leser inn data
gdf = gpd.read_file(fgb_file)

# Maks antall rader som skal strømmes
max_rader = 30

# Begrens datasettet til maks 30 rader
gdf = gdf.iloc[:max_rader]

# Definer batch-størrelse
batch_size = 10

print("Starter simulert strømming av AR50-data...\n")

# Strømmer data i batches
for i in range(0, len(gdf), batch_size):
    batch = gdf.iloc[i:i+batch_size]
    print(batch)
    print(f"Strømmet {min(i+batch_size, len(gdf))} av {len(gdf)} rader...\n")
    time.sleep(2)

print("Strømming fullført!")



Starter simulert strømming av AR50-data...

                                               kilde  artype  arskogbon  \
0  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      99         98   
1  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      99         98   
2  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      99         98   
3  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      82         98   
4  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      50         98   
5  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      50         98   
6  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      50         98   
7  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      50         98   
8  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      50         98   
9  AR50 fra AR5 årsversjon 2021. ARFJELL2 og N50 ...      50         98   

   artreslag  arjordbr  arveget arkartstd  \
0         98        98       98      AR50   
1         98        98       98      AR5

## Romlig spørring - bounding box

## Bruker bounding box til å bestemme en bestemt lokasjon man ønsker informasjon og filrtrere kolonnene etterbehov og hvilken verdi man vi se

In [95]:


def create_map(ar50, column, value, minx=7.95, miny=58.08, maxx=8.15, maxy=58.20):
    """
    Oppretter et Folium-kart med filtrert AR50-data basert på en gitt kolonne og verdi.

    :param ar50: GeoDataFrame med AR50-data
    :param column: Kolonnen som skal filtreres etter (f.eks. 'artreslag')
    :param value: Verdien som skal filtreres (f.eks. 99)
    :param minx, miny, maxx, maxy: Bounding box-koordinater (standard er Kristiansand)
    :return: Folium-kart
    """
    # Definer bounding box
    bbox = box(minx, miny, maxx, maxy)

    # Forsikre deg om at CRS matcher
    ar50 = ar50.to_crs(epsg=4326)
    bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

    # Filtrer AR50-data basert på kolonne og verdi
    ar50_filtered = ar50[ar50[column] == value]

    # Klipp data til bounding box
    ar50_clipped = gpd.clip(ar50_filtered, bbox_gdf)

    # Finn senter av bounding box for å plassere kartet
    center = [(miny + maxy) / 2, (minx + maxx) / 2]

    # Lag Folium-kart
    m = folium.Map(location=center, zoom_start=10, tiles="cartodb positron")

    # Legg til bounding box som en blå firkant
    bbox_coords = [(miny, minx), (miny, maxx), (maxy, maxx), (maxy, minx), (miny, minx)]
    folium.Polygon(bbox_coords, color="blue", weight=2, fill=False).add_to(m)

    # Legg til AR50-data med valgte verdier i kartet
    folium.GeoJson(
        ar50_clipped,
        name=f"AR50 - {column} = {value}",
        style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.5},
    ).add_to(m)

    # Legg til en "Layer Control" for å slå lag av/på
    folium.LayerControl().add_to(m)

    return m

# Last inn AR50-data (endre filstien til riktig sti)
ar50 = gpd.read_file("/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb")

# Kall funksjonen med ønsket kolonne og verdi
m = create_map(ar50, column="artreslag", value=99)

# Vis kartet
m


## Bruker buffer for å begrense datasettet

In [117]:


def create_map_with_buffer(ar50, column, value, center_x, center_y, buffer_radius):
    """
    Oppretter et Folium-kart med AR50-data filtrert etter en buffer rundt et gitt punkt.

    :param ar50: GeoDataFrame med AR50-data
    :param column: Kolonnen som skal filtreres etter (f.eks. 'artreslag')
    :param value: Verdien som skal filtreres (f.eks. 99)
    :param center_x, center_y: Koordinatene til senterpunktet (longitude, latitude)
    :param buffer_radius: Radius i meter for bufferen rundt punktet
    :return: Folium-kart
    """
    # Forsikre deg om at CRS matcher (EPSG:4326 for kart, men buffer krever meter-projeksjon)
    ar50 = ar50.to_crs(epsg=25832)  # EPSG:25832 = UTM-sone for Sør-Norge (meterbasert)

    # Lag et sentralt punkt (i UTM-koordinater)
    center_point = Point(center_x, center_y)

    # Konverter punktet til riktig CRS
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]

    # Lag en buffer (radius i meter)
    buffer_geom = center_point.buffer(buffer_radius)

    # Konverter bufferen tilbake til EPSG:4326 for kartvisning
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Filtrer AR50-data på ønsket kolonne og verdi
    ar50_filtered = ar50[ar50[column] == value]

    # Filtrer AR50-data som ligger innenfor bufferen
    ar50_clipped = ar50_filtered[ar50_filtered.geometry.within(buffer_geom)]

    # Finn senter for kartvisning
    center = [center_y, center_x]

    # Lag Folium-kart
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.0},
    ).add_to(m)

    # Legg til AR50-data innenfor buffer
    folium.GeoJson(
        ar50_clipped,
        name=f"AR50 - {column} = {value}",
        style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.5},
    ).add_to(m)

    # Legg til en "Layer Control" for å slå lag av/på
    folium.LayerControl().add_to(m)

    return m

# Last inn AR50-data
ar50 = gpd.read_file("/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb")

# Definer koordinater for sentrumspunktet (Kristiansand sentrum)
center_x, center_y = 8.005, 58.146  # Long, Lat

# Kall funksjonen for å lage kart med buffer (radius på 5000 meter) Endre verdier etter ønsnke
m = create_map_with_buffer(ar50, column="arskogbon", value=13, center_x=center_x, center_y=center_y, buffer_radius=5000)

# Vis kartet
m


## Med partial read

In [120]:

def create_map_with_buffer_partialread(fgb_path, column, value, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    """
    #LESER KUN VALGT KOLONNE + GEOMETRI
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Filtrering
    gdf_filtered = gdf[gdf[column] == value]
    gdf_clipped = gdf_filtered[gdf_filtered.geometry.within(buffer_geom)]

    # Kart
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til buffer
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.0},
    ).add_to(m)

    # Legg til AR50-data
    folium.GeoJson(
        gdf_clipped.to_crs(epsg=4326),
        name=f"{column} = {value}",
        style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.5},
    ).add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gdb.fgb"
m = create_map_with_buffer_partialread(
    fgb_path=fgb_file,
    column="arskogbon",
    value=13,
    center_x=8.005,
    center_y=58.146,
    buffer_radius=5000
)
m


## Bruker partial read for å lese filen, så lastes ned den nyfiltrerte.  "bruker partial read med pyogrio.read_dataframe() for å lese kun den kolonnen du skal filtrere på + geometrien."

In [90]:


def sjekk_filstørrelse(filsti):
    """Returnerer filstørrelse i MB eller KB."""
    if not os.path.exists(filsti):
        return "Fil ikke funnet"
    størrelse = os.path.getsize(filsti)
    return f"{størrelse / (1024 * 1024):.2f} MB" if størrelse > 100_000 else f"{størrelse / 1024:.2f} KB"

def les_fil_med_tid(filsti, kolonne=None):
    """Leser GeoDataFrame med tid, støtter partial read ved spesifikk kolonne."""
    start = time.time()

    if kolonne:
        gdf = pyogrio.read_dataframe(filsti, columns=[kolonne])  # Kun én kolonne + geometri
    else:
        gdf = gpd.read_file(filsti)  # Fallback til vanlig lesing

    slutt = time.time()
    lesetid = slutt - start
    return gdf, lesetid

def filtrer_og_sammenlign(input_fgb, kolonne, verdi, output_fgb):
    print(f"Leser originalfil (kun kolonne '{kolonne}') fra: {input_fgb}")
    gdf_org, tid_org = les_fil_med_tid(input_fgb, kolonne=kolonne)

    print(f"\nFiltrerer på: {kolonne} = {verdi}")
    gdf_filtrert = gdf_org[gdf_org[kolonne] == verdi]

    if gdf_filtrert.empty:
        print("Ingen treff på filteret!")
        return

    print(f"Lagrer filtrert data til: {output_fgb}")
    gdf_filtrert.to_file(output_fgb, driver="FlatGeobuf")

    # Les den nye filen (hele, ikke partial)
    gdf_ny, tid_ny = les_fil_med_tid(output_fgb)

    # Sammenligning
    print("\n Sammenligning:")
    print(f"- Original filstørrelse: {sjekk_filstørrelse(input_fgb)}")
    print(f"- Filtrert filstørrelse: {sjekk_filstørrelse(output_fgb)}")
    print(f"- Tid å lese originalfil: {tid_org:.2f} sekunder (med partial read)")
    print(f"- Tid å lese filtrert fil: {tid_ny:.2f} sekunder")
    print(f"- Antall rader etter filtrering: {len(gdf_filtrert)}")

# Parametere
input_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gdb.fgb"
output_fgb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/filtered_arjordbr_24.fgb"
kolonne = "arjordbr"
verdi = 24

In [89]:

# Kjør funksjonen
filtrer_og_sammenlign(input_fgb, kolonne, verdi, output_fgb)


Leser originalfil (kun kolonne 'arjordbr') fra: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gdb.fgb

Filtrerer på: arjordbr = 24
Lagrer filtrert data til: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/filtered_arjordbr_24.fgb

 Sammenligning:
- Original filstørrelse: 275.65 MB
- Filtrert filstørrelse: 5.54 MB
- Tid å lese originalfil: 2.24 sekunder (med partial read)
- Tid å lese filtrert fil: 0.01 sekunder
- Antall rader etter filtrering: 3807


## Filtrering "Jeg vil se hvilke treslag jeg har på et område innen 500m "

In [114]:

def create_map_with_buffer_partialread(fgb_path, column, value, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    """
    #LESER KUN VALGT KOLONNE + GEOMETRI
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Filtrering
    gdf_filtered = gdf[gdf[column] == value]
    gdf_clipped = gdf_filtered[gdf_filtered.geometry.within(buffer_geom)]

    # Kart
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til buffer
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.2},
    ).add_to(m)

    # Legg til AR50-data
    folium.GeoJson(
        gdf_clipped.to_crs(epsg=4326),
        name=f"{column} = {value}",
        style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.5},
    ).add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
m = create_map_with_buffer_partialread(
    fgb_path=fgb_file,
    column="artreslag",
    value=31,
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=500
)
m


In [105]:
import pyogrio
import geopandas as gpd
import folium
from shapely.geometry import Point
from IPython.display import display

def create_map_with_buffer_partialread_all(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle verdier i den angitte kolonnen (f.eks. 'artreslag') innenfor bufferområdet.
    """
    # Les kun valgt kolonne + geometri
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    # Sentrer input er i EPSG:4326, projiser til UTM
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut alle objekter som ligger innenfor bufferområdet
    gdf_clipped = gdf[gdf.geometry.within(buffer_geom)]

    # Kart (bruker senterpunkt i EPSG:4326)
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.2},
    ).add_to(m)

    # Legg til AR50-data med tooltip som viser verdien i den angitte kolonnen
    # Konverterer dataene tilbake til EPSG:4326
    geojson_layer = folium.GeoJson(
        gdf_clipped.to_crs(epsg=4326),
        name="AR50-data",
        style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.5},
        tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
    )
    geojson_layer.add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
m = create_map_with_buffer_partialread_all(
    fgb_path=fgb_file,
    column="artreslag",
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=1000
)
display(m)


In [130]:
import pyogrio
import geopandas as gpd
import folium
from shapely.geometry import Point
from IPython.display import display

def create_map_with_buffer_partialread_all(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle verdier i den angitte kolonnen (f.eks. 'artreslag') innenfor bufferområdet,
    med hvert unike område fargekodet etter verdien i kolonnen.
    """
    # Les kun valgt kolonne + geometri
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    # Sentrer input er i EPSG:4326, projiser til UTM
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut alle objekter som ligger innenfor bufferområdet
    gdf_clipped = gdf[gdf.geometry.within(buffer_geom)]

    # Bygg et fargekart basert på unike verdier i den angitte kolonnen
    unique_values = gdf_clipped[column].unique().tolist()
    # Liste over farger – utvid eller juster denne listen ved behov
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Lag kart med senterpunkt (i EPSG:4326)
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.2},
    ).add_to(m)

    # Funksjon for å style hvert objekt basert på verdien i den angitte kolonnen
    def style_func(feature):
        val = feature['properties'][column]
        return {"color": color_map.get(val, "black"), "weight": 1, "fillOpacity": 0.5}

    # Legg til AR50-data som GeoJson med tooltip
    geojson_layer = folium.GeoJson(
        gdf_clipped.to_crs(epsg=4326),
        name="AR50-data",
        style_function=style_func,
        tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
    )
    geojson_layer.add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
m = create_map_with_buffer_partialread_all(
    fgb_path=fgb_file,
    column="artreslag",
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=500

)
display(m)


Fargekart: {31: 'red', 33: 'blue'}


In [110]:
from shapely import wkt
import folium
import shapely.wkt
import pandas as pd
import duckdb


con = duckdb.connect()
con.execute("INSTALL spatial; LOAD spatial;")

query = f"""
    SELECT
        "områdeId",
        "arealtype",
        ST_AsText("geom") AS geometry_wkt
    FROM ST_Read('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/output_gml.fgb')
    WHERE "områdeID" = 4204
    LIMIT 10
"""

result = con.execute(query).fetchdf()
display(result)
result['geometry'] = result['geometry_wkt'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(result, geometry='geometry')
gdf = gdf.set_crs("EPSG:25832", allow_override=True)

gdf_projected = gdf.to_crs("EPSG:3857")

center_point_projected = gdf_projected.geometry.union_all().centroid

center_point = gpd.GeoSeries([center_point_projected], crs="EPSG:3857").to_crs("EPSG:4326").iloc[0]
center_lat = center_point.y
center_lon = center_point.x

gdf = gdf.to_crs("EPSG:4326")

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)


def swap_coords(coords):
    return [(lat, lon) for lon, lat in coords]

for idx, row in gdf.iterrows():
    geom = row['geometry']
    if geom.geom_type == 'Polygon':
        coords = list(geom.exterior.coords)
        m.add_child(folium.Polygon(locations=swap_coords(coords),
                                   color='blue', fill=True, fill_opacity=0.5))

m

,områdeId,arealtype,geometry_wkt
0,4204,99,"POLYGON ((455377.855658909 6416267.628298172, ..."
1,4204,82,"POLYGON ((459409.0391476797 6418114.440908768,..."
2,4204,50,"POLYGON ((451923.6871628836 6442360.380846969,..."
3,4204,50,POLYGON ((449901.37398613873 6441050.035043333...
4,4204,30,"POLYGON ((450633.4404859477 6442838.866626649,..."
5,4204,30,POLYGON ((450348.88713862223 6442322.468589208...
6,4204,30,"POLYGON ((450485.7496845983 6442585.193880714,..."
7,4204,30,POLYGON ((450456.27949738654 6442803.006089125...
8,4204,30,"POLYGON ((450318.7061109386 6442660.45893839, ..."
9,4204,30,"POLYGON ((450077.4618917754 6442668.976943247,..."


In [131]:


def create_map_with_buffer_partialread_all(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle verdier i den angitte kolonnen (f.eks. 'artreslag') innenfor bufferområdet,
    med hvert unike område fargekodet etter verdien i kolonnen. Objekter som bare delvis
    ligger innenfor bufferen, klippes slik at kun den delen innenfor bufferen vises.
    """
    # Les kun valgt kolonne + geometri
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    # Sentrer input er i EPSG:4326, projiser til UTM
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut (clip) objekter slik at vi får med den delen som ligger innenfor bufferen,
    # selv om objektet ikke er helt innenfor.
    gdf_clipped = gpd.clip(gdf, buffer_geom)

    # Bygg et fargekart basert på unike verdier i den angitte kolonnen
    unique_values = gdf_clipped[column].unique().tolist()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Lag kart med senterpunkt (i EPSG:4326)
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.2},
    ).add_to(m)

    # Funksjon for å style hvert objekt basert på verdien i den angitte kolonnen
    def style_func(feature):
        val = feature['properties'][column]
        return {"color": color_map.get(val, "black"), "weight": 1, "fillOpacity": 0.5}

    # Legg til AR50-data som GeoJson med tooltip. Dataene konverteres tilbake til EPSG:4326.
    geojson_layer = folium.GeoJson(
        gdf_clipped.to_crs(epsg=4326),
        name="AR50-data",
        style_function=style_func,
        tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
    )
    geojson_layer.add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
m = create_map_with_buffer_partialread_all(
    fgb_path=fgb_file,
    column="artreslag",
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=500
)
display(m)


Fargekart: {98: 'red', 31: 'blue', 33: 'green', 39: 'orange'}


In [133]:


def create_map_with_buffer_partialread_all(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle verdier i den angitte kolonnen (f.eks. 'artreslag') innenfor bufferområdet,
    med hvert unike område fargekodet etter verdien i kolonnen. Objekter som bare delvis
    ligger innenfor bufferen, klippes slik at kun den delen innenfor bufferen vises.
    """
    # Les kun valgt kolonne + geometri
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    # Sentrer input er i EPSG:4326, projiser til UTM
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut (clip) objekter slik at vi får med den delen som ligger innenfor bufferen,
    # selv om objektet ikke er helt innenfor.
    gdf_clipped = gpd.clip(gdf, buffer_geom)

    # Bygg et fargekart basert på unike verdier i den angitte kolonnen
    unique_values = gdf_clipped[column].unique().tolist()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Lag kart med senterpunkt (i EPSG:4326)
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.1},
    ).add_to(m)

    # Funksjon for å style hvert objekt basert på verdien i den angitte kolonnen
    def style_func(feature):
        val = feature['properties'][column]
        return {"color": color_map.get(val, "black"), "weight": 1, "fillOpacity": 0.4}

    # Legg til AR50-data som GeoJson med tooltip. Dataene konverteres tilbake til EPSG:4326.
    geojson_layer = folium.GeoJson(
        gdf_clipped.to_crs(epsg=4326),
        name="AR50-data",
        style_function=style_func,
        tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
    )
    geojson_layer.add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
m = create_map_with_buffer_partialread_all(
    fgb_path=fgb_file,
    column="artreslag",
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=500
)
display(m)
### De uregisterte feltene vises som vedi 98? og vises som røde felt.

Fargekart: {98: 'red', 31: 'blue', 33: 'green', 39: 'orange'}


In [134]:
import pyogrio
import geopandas as gpd
import folium
from shapely.geometry import Point
from IPython.display import display

def create_map_with_buffer_partialread_all(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle objekter innenfor bufferområdet med fargekoding etter verdien i 'column'.
    Objekter som kun delvis ligger innenfor bufferen klippes slik at bare den delen som ligger
    innenfor bufferen vises. I tillegg lages separate lag for hver unik verdi slik at de kan
    slås av/på via LayerControl.
    """
    # Les kun valgt kolonne + geometri
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for korrekt buffer (meterbasert)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    # Sentrer input er i EPSG:4326, projiser til UTM
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut (clip) objekter slik at vi får med den delen som ligger innenfor bufferen,
    # selv om objektet ikke er helt innenfor.
    gdf_clipped = gpd.clip(gdf, buffer_geom)

    # Bygg et fargekart basert på unike verdier i den angitte kolonnen
    unique_values = gdf_clipped[column].unique().tolist()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Lag kart med senterpunkt (i EPSG:4326)
    center = [center_y, center_x]
    m = folium.Map(location=center, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "blue", "weight": 2, "fillOpacity": 0.1},
    ).add_to(m)

    # For hvert unike verdi i kolonnen, opprett et lag med de tilhørende objektene
    for val in unique_values:
        subset = gdf_clipped[gdf_clipped[column] == val].to_crs(epsg=4326)
        # Her fanger vi den nåværende 'val' i lambdaens standardparameter
        layer = folium.GeoJson(
            subset,
            name=f"{column}: {val}",
            style_function=lambda feature, val=val: {
                "color": color_map.get(val, "black"),
                "weight": 1,
                "fillOpacity": 0.4
            },
            tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
        )
        layer.add_to(m)

    folium.LayerControl().add_to(m)
    return m

# Eksempelbruk
fgb_file = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50_FDG/NY_output.fgb"
m = create_map_with_buffer_partialread_all(
    fgb_path=fgb_file,
    column="artreslag",   # Her kan du endre til "artreslag" eller en annen kolonne
    center_x=8.01122450699552,
    center_y=58.1743128719827,
    buffer_radius=500
)
display(m)


Fargekart: {98: 'red', 31: 'blue', 33: 'green', 39: 'orange'}
